In [ ]:
from __future__ import annotations
from dataclasses import asdict
from pathlib import Path
import json
import numpy as np

from src.extract_image_metrics import *

from src.build_profile import build_profile

In [2]:
train_dataset = Path(r"datasets\vaca\images")
output_json = Path(r"data\output")

In [ ]:
build_profile(train_dataset, output_json)

In [40]:
profile = Path(r"data\output\profile.json")
image_folder = Path(r"datasets\vaca\images\train")

with profile.open("r", encoding="utf-8") as f:
    data = json.load(f)

In [41]:
import pandas as pd

rows = []

for img in image_folder.rglob("*"):
    if img.suffix.lower() in {".jpg", ".jpeg", ".png", ".webp"}:
        metrics = extract_metrics(img)
        result = classify(metrics, data)
        status, score, details = result

        row = {
            "file": img.name,
            "status": status,
            "score": score,
            "hard_fail": details["hard_fail"],
            "warn": details["warn"],
        }

        for k, v in details["z"].items():
            row[f"z_{k}"] = v

        rows.append(row)

df = pd.DataFrame(rows)


In [42]:
df

,file,status,score,hard_fail,warn,z_brightness_mean,z_brightness_std,z_contrast_std,z_sharpness_lap_var,z_pct_black,z_pct_white,z_entropy
0,0716_DJI_JPG.rf.48ec98fe23ec5d7a0bbd76514c0928...,PASS,1.959464,[],[],-1.205632,-0.897127,-0.897127,-0.549158,-0.396463,-0.226450,-0.515985
1,315A_2_jpg.rf.7bd5d69ecffd72f5b319c4972c9274e4...,PASS,2.299189,[],[],0.360029,0.397357,0.397357,2.071599,-0.220240,-0.224674,0.671084
2,315A_jpg.rf.015a1a54af02a1f5f465c851a1254c11.jpg,PASS,2.543306,[],[],-0.801001,1.165640,1.165640,1.269966,-0.082527,0.020354,1.220382
3,315B_jpg.rf.8f5386cea2a29cf4b1706575afef00d5.jpg,PASS,3.046102,[],[],-1.074458,1.442858,1.442858,1.480898,-0.216147,0.091740,1.308590
4,325A_jpg.rf.ce7f1b609532d4f6ccd9987ebbcacb57.jpg,PASS,1.771847,[],[],0.091280,1.064601,1.064601,-0.129916,0.422573,-0.179420,0.797948
...,...,...,...,...,...,...,...,...,...,...,...,...
792,PASTO-37D-PAST-1_JPG.rf.813d42812c54ace53b2c1c...,PASS,2.559440,[],[],1.138539,-1.393476,-1.393476,-0.335393,-0.400623,-0.111935,-1.041823
793,R1_JPG.rf.e690ee296440ba14ca3676a3db5124d1.jpg,PASS,0.839583,[],[],0.388891,0.101196,0.101196,-0.424313,-0.237783,0.190593,0.510171
794,R2_JPG.rf.b051c62fe3b4e987ef37dd358843e5c7.jpg,PASS,0.694366,[],[],-0.044026,0.042702,0.042702,-0.450673,-0.283014,0.286923,0.333214
795,vjbaia4_jpg.rf.2a6efee00444e53075b02b3bc3c4541...,PASS,2.853497,[],[],-1.031964,1.223031,1.223031,1.619094,0.201865,0.046674,1.192265


In [43]:
df.to_excel(r"data\output\image_score.xlsx", index=False)

In [46]:
import cv2
from pathlib import Path

def resize_image(input_path: str, output_path: str, width: int, height: int):
    input_path = Path(input_path)
    output_path = Path(output_path)

    # Lê a imagem
    image = cv2.imread(str(input_path))

    if image is None:
        raise ValueError("Imagem não encontrada ou inválida.")

    # Redimensiona
    resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    # Cria pasta se não existir
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Salva imagem
    cv2.imwrite(str(output_path), resized)

    print(f"Imagem salva em: {output_path}")
    print(f"Nova resolução: {width}x{height}")


# Exemplo de uso



In [47]:
image_folder = Path(r"datasets\vaca\images\train")
output_image_folder = Path(r"datasets\vaca_pequena")

for img in image_folder.rglob("*"):
    if img.suffix.lower() in {".jpg", ".jpeg", ".png", ".webp"}:
        output_path = Path.joinpath(output_image_folder, img.name)
        resize_image(
            input_path=img,
            output_path=output_path,
            width=640,
            height=640
        )
        

Imagem salva em: datasets\vaca_pequena\0716_DJI_JPG.rf.48ec98fe23ec5d7a0bbd76514c092868.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\315A_2_jpg.rf.7bd5d69ecffd72f5b319c4972c9274e4.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\315A_jpg.rf.015a1a54af02a1f5f465c851a1254c11.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\315B_jpg.rf.8f5386cea2a29cf4b1706575afef00d5.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\325A_jpg.rf.ce7f1b609532d4f6ccd9987ebbcacb57.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\325B_jpg.rf.b21e766f6d4debd5a0f501410964792d.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\326A_2_jpg.rf.5b31e02e9a16d35d2f7e17e3331d4176.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\326A_jpg.rf.a59b96586c4be282c2de8a713a62afe0.jpg
Nova resolução: 640x640
Imagem salva em: datasets\vaca_pequena\326_jpg.rf.e619eef550772d2891b5071d5fce3d5f.jpg
Nova reso